In [ ]:
from rag_chatbot.core.settings import settings
from rag_chatbot.rag.query_embedder import QueryEmbedder
from rag_chatbot.rag.retriever import Retriever
from rag_chatbot.rag.llm import get_llm
from rag_chatbot.prompt.prompts import get_prompt
from rag_chatbot.rag.pipeline import RAGPipeline
from rag_chatbot.rag.evaluation import build_evaluation_table

d:\10Acadamy\Week 7\Tasks\rag-complaint-chatbot\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
persist_path = settings.paths.VECTOR_STORE["fiass_dir"]
index_path = persist_path/"faiss.index"
metadata_path = persist_path/"metadata.parquet"

In [ ]:
embedder = QueryEmbedder()

In [ ]:
retriever = Retriever(
    index_path=  index_path,
    metadata_path= metadata_path,
    k=5
)

In [ ]:
llm = get_llm()

d:\10Acadamy\Week 7\Tasks\rag-complaint-chatbot\.venv\lib\site-packages\huggingface_hub\file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Loading Mistral via CTransformers from: D:\10Acadamy\Week 7\Tasks\rag-complaint-chatbot\models\tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf


In [ ]:
prompt = get_prompt()

In [ ]:
rag = RAGPipeline(
    embedder=embedder,
    retriever=retriever,
    llm=llm,
    prompt=prompt,
)

In [ ]:
query = "What are the most common complaints about credit cards?"
result = await rag.arun(query)
result

{'query': 'What are the most common complaints about credit cards?',
 'answer': 'Answer: The most common complaints about credit cards are fraudulent charges, unauthorized transactions, and incorrect billing information.',
 'confidence': 0.73,
 'sources': [{'document': 'any. i am sure that you had many other complaints. i would like to get my money back and hope you will restrict them from operating a credit card scam.',
   'complaint_id': '6604272',
   'product_category': 'Credit Card',
   'product': 'Credit card or prepaid card',
   'issue': 'Closing your account',
   'sub_issue': "Can't close your account",
   'company': 'Self Financial Inc.',
   'state': 'VA',
   'date_received': '2023-02-22',
   'chunk_index': 2,
   'total_chunks': 3,
   'score': 0.7350383400917053},
  {'document': 'ompetence of the bank of america credit card team. i have used many credit cards, and i have never encountered such issues, and have to talk to clueless representatives over and over again.',
   'compl

In [ ]:
import asyncio
queries = [
    "What are common credit card complaints?",
    "Why do customers complain about money transfers?",
    "What issues do users report with cryptocurrency fees?",
]


# 1. Create the list of coroutines
tasks = [rag.arun(q) for q in queries]

# 2. Use 'gather' to run them all and wait for the actual DICTIONARY results
# Note: In a Jupyter Notebook, you can use 'await' directly
results = await asyncio.gather(*tasks)

# 3. Now 'results' contains real data, so this will work
df_eval = build_evaluation_table(results)

### Evaluation Summary

- **Precision@5** measures how many retrieved chunks are truly relevant.
- **Faithfulness** measures how grounded the answer is in retrieved context.
- Low faithfulness indicates hallucination risk.
- Low precision suggests retriever tuning is needed (MMR, filters, metadata).
